In [1]:
import pandas as pd
import numpy as np

# Select Transactions from DB

In [7]:
import psycopg2 
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os


load_dotenv()


hostname=os.getenv("hostname")
database=os.getenv("database")
username=os.getenv("username")
pwd=os.getenv("pwd")
port_id=os.getenv("port_id")
host = os.getenv("host")


In [8]:
conn_string = f"postgresql://{username}:{pwd}@{host}:{port_id}/{database}"
db =  create_engine(url=conn_string)
conn=db.connect()

df = pd.read_sql("SELECT * FROM transactions", db)

df.head()



,transaction_date,owner,ticker,asset_description,asset_type,type,amount,comment,senator,ptr_link,disclosure_date,id
0,10/28/2022,Joint,MSFT,Microsoft Corporation - Common Stock Option Ty...,Stock Option,Sale (Full),"$1,001 - $15,000",--,Thomas H Tuberville,https://efdsearch.senate.gov/search/view/ptr/b...,11/10/2022,0
1,10/31/2022,Joint,CLF,Cleveland-Cliffs Inc. Common Stock,Stock,Purchase,"$15,001 - $50,000",--,Thomas H Tuberville,https://efdsearch.senate.gov/search/view/ptr/b...,11/10/2022,1
2,10/28/2022,Joint,MSFT,Microsoft Corporation - Common Stock Option Ty...,Stock Option,Sale (Full),"$1,001 - $15,000",--,Thomas H Tuberville,https://efdsearch.senate.gov/search/view/ptr/b...,11/10/2022,2
3,10/28/2022,Joint,MSFT,Microsoft Corporation - Common Stock Option Ty...,Stock Option,Sale (Full),"$1,001 - $15,000",--,Thomas H Tuberville,https://efdsearch.senate.gov/search/view/ptr/b...,11/10/2022,3
4,10/24/2022,Joint,MSFT,Microsoft Corporation - Common Stock,Stock,Purchase,"$15,001 - $50,000",--,Thomas H Tuberville,https://efdsearch.senate.gov/search/view/ptr/b...,11/10/2022,4


# Gather Tickers and Transaction Dates

In [10]:
dateSer = df.groupby("ticker")['transaction_date'].min()
dateSer.head()

ticker
--         01/01/2014
0QZI.IL    09/25/2019
3V64.TI    09/25/2019
A          02/24/2015
AA         01/06/2021
Name: transaction_date, dtype: object

In [12]:
allTickers = list(dateSer.index)
allTickers[:5]

['--', '0QZI.IL', '3V64.TI', 'A', 'AA']

In [24]:
apiKey = os.getenv('apiKey')
tick = 'KO'

testEndpoint =f"https://www.alphavantage.co/query?function=TIME_SERIES_WEEKLY&symbol={tick}&apikey={apiKey}"

print(testEndpoint)


https://www.alphavantage.co/query?function=TIME_SERIES_WEEKLY&symbol=KO&apikey=None


In [23]:
# testEndpoint = f"https://www.alphavantage.co/query?function=TIME_SERIES_WEEKLY&symbol=IBM&apikey=U288LAEMR485UKD1"
